# Fetch data

In [1]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [2]:
ground_truth[0] 

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

# Evaluating retrieval

In [4]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# Q1. Minsearch text

Now let's evaluate our usual minsearch approach, but tweak the parameters. Let's use the following boosting params:

```
boost = {'question': 1.5, 'section': 0.1}
```
What's the hit rate for this approach?


In [5]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [6]:

def minsearch_search(query, course, boost):
    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

metrics = evaluate(ground_truth=ground_truth, search_function=lambda q: minsearch_search(q['question'], q['course'], boost = {'question': 1.5, 'section': 0.1}))
print(metrics)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.848714069591528, 'mrr': 0.7288235717887772}


# Embeddings

---
The latest version of minsearch also supports vector search. We will use it:


```python
from minsearch import VectorSearch
```

We will also use TF-IDF and Singular Value Decomposition to create embeddings from texts. You can refer to our "Create Your Own Search Engine" workshop if you want to know more about it.

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
```
Let's create embeddings for the "question" field:


```python
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)
```

In [7]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline


In [8]:
texts_q2  = []

for doc in documents:
    t = doc['question']
    texts_q2.append(t)

pipeline_q2 = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline_q2.fit_transform(texts_q2)

# Q2. Vector search for question

---

Now let's index these embeddings with minsearch:


```python
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

```
Evaluate this seach method. What's MRR for it?



In [9]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [10]:

def search_q2(q):
    # 取得查詢向量
    query_vec = pipeline_q2.transform([q['question']]) # (1,128)
    return vindex.search(
        query_vector=query_vec, 
        filter_dict={'course': q['course']},
        num_results=5
    )
# 執行評估
metrics_q2 = evaluate(ground_truth, search_q2)
print(metrics_q2)


  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.48173762697212014, 'mrr': 0.3571284489590088}


# Q3. Vector search for question and answer

---
We only used question in Q2. We can use both question and answer:


```python
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)
```

Using the same pipeline (min_df=3 for TF-IDF vectorizer and n_components=128` for SVD), evaluate the performance of this approach

What's the hitrate?

In [11]:
texts_q3  = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts_q3.append(t)

pipeline_q3 = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline_q3.fit_transform(texts_q3)

In [12]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [13]:
def search_function(q):
    # 取得查詢向量
    query_vec = pipeline_q3.transform([q['question']])
    return vindex.search(
        query_vector=query_vec, 
        filter_dict={'course': q['course']},
        num_results=5
    )

# 執行評估
metrics = evaluate(ground_truth, search_function)
print(metrics)


  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.8210503566025502, 'mrr': 0.6717707657949719}


# Q4. Qdrant
---
Now let's evaluate the following settings in Qdrant:
```
text = doc['question'] + ' ' + doc['text']
model_handle = "jinaai/jina-embeddings-v2-small-en"
limit = 5
```
What's the MRR?

In [14]:
from qdrant_client import QdrantClient, models
import numpy as np
from tqdm import tqdm

# 文字嵌入模型初始化

EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

# Qdrant 客戶端
client = QdrantClient("http://localhost:6333")



In [15]:

collection_name = "q4_evaluation"

client.delete_collection(collection_name=collection_name)
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE)
)

client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

# 建立向量並上傳
points = []
for i, doc in tqdm(enumerate(documents)):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)
    

client.upsert(collection_name=collection_name, points=points)


948it [00:00, 92148.32it/s]


UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
def search_q4(q, limit: int = 5):
    # print(q)

    ret = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=q["question"],
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=q["course"])
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in ret.points:
        results.append(point.payload)
    
    return results

    
metrics_q4 = evaluate(ground_truth, search_q4)
print(metrics_q4)


100%|██████████| 4627/4627 [00:32<00:00, 140.44it/s]

{'hit_rate': 0.9299762264966501, 'mrr': 0.8517722066133576}


In [17]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

# Q5. Cosine simiarity

---

In the second part of the module, we looked at evaluating the entire RAG approach. In particular, we looked at comparing the answer generated by our system with the actual answer from the FAQ.

One of the ways of doing it is using the cosine similarity. Let's see how to calculate it.

Cosine similarity is a dot product between two normalized vectors. In geometrical sense, it's the cosine of the angle between the vectors. Look up "cosine similarity geometry" if you want to learn more about it.

For us, it means that we need two things:

- First, we normalize each of the vectors
- Then, compute the dot product
So, we get this:

```python=
def cosine(u, v):
    u = normalize(u)
    v = normalize(v)
    return u.dot(v)
```
For normalization, we first compute the vector norm (its length), and then divide the vector by it:

```python=
def normalize(u):
    norm = np.sqrt(u.dot(u))
    return u / norm
```
(where np is import numpy as np)

Or we can simplify it:

```python=
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)
```

Now let's use this function to compute the A->Q->A cosine similarity.

We will use the results from our gpt-4o-mini evaluations:

```python=
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)
```
When creating embeddings, we will use a simple way - the same we used in the Embeddings section:

```python=
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
```

Let's fit the vectorizer on all the text data we have:

pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)
Now use the transform methon of the pipeline to create the embeddings and calculate the cosine similarity between each pair.

What's the average cosine?

This is how you do it:

- For each answer pair, compute
    - v_llm for the answer from the LLM
    - v_orig for the original answer
    - then compute the cosine between them
- At the end, take the average

In [18]:
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=3)),
                ('truncatedsvd',
                 TruncatedSVD(n_components=128, random_state=1))])

In [19]:
df_results.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [20]:
pipeline_rag = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

pipeline_rag.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=3)),
                ('truncatedsvd',
                 TruncatedSVD(n_components=128, random_state=1))])

In [21]:
cosines = []
for _, row in df_results.iterrows():
    v1 = pipeline_rag.transform([row['answer_llm']])[0]
    v2 = pipeline_rag.transform([row['answer_orig']])[0]
    cosines.append(cosine(v1, v2))
print("Q5 avg cosine:", np.mean(cosines))

Q5 avg cosine: 0.8415841233490402


In [22]:
df_results['cosine'] = cosines
df_results['cosine'].describe()

count    1830.000000
mean        0.841584
std         0.173737
min         0.079093
25%         0.806927
50%         0.905812
75%         0.950711
max         0.996457
Name: cosine, dtype: float64

# Q6. Rouge
---
And alternative way to see how two texts are similar is ROUGE.

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:
```shell=
pip install rouge
```
(The latest version at the moment of writing is 1.0.1)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)
```python=
from rouge import Rouge
rouge_scorer = Rouge()

r = df_results.iloc[10]
scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
scores
```
There are three scores: rouge-1, rouge-2 and rouge-l, and precision, recall and F1 score for each.

- rouge-1 - the overlap of unigrams,
- rouge-2 - bigrams,
- rouge-l - the longest common subsequence
For the 10th document, Rouge-1 F1 score is 0.45

Let's compute it for the pairs in the entire dataframe. What's the average Rouge-1 F1?

In [23]:
from rouge import Rouge
rouge = Rouge()

scores = []
for _, row in df_results.iterrows():
    sc = rouge.get_scores(row['answer_llm'], row['answer_orig'])[0]['rouge-1']['f']
    scores.append(sc)
print("avg Rouge-1 F1:", np.mean(scores))


avg Rouge-1 F1: 0.3516946452113943
